# Finetuning data: 사전학습과 기초 준비 비교하기

In [ ]:
import jsonlines
import itertools
import pandas as pd
from pprint import pprint

import datasets
from datasets import load_dataset

## 사전학습 데이터셋 확인하기

**주의**, "The Pile" 데이터셋은 현재 이전중이므로 강의에서와 동일한 예시를 볼 수 없습니다.  
대신 다른 데이터셋 ["Common Crwal"](https://huggingface.co/datasets/c4)을 이용해보세요.

In [ ]:
#pretrained_dataset = load_dataset("EleutherAI/pile", split="train", streaming=True)

pretrained_dataset = load_dataset("c4", "en", split="train", streaming=True)

딕셔너리 형태의 객체가 출력될 것입니다

In [ ]:
n = 5
print("Pretrained dataset:")
top_n = itertools.islice(pretrained_dataset, n)
for i in top_n:
  print(i)

### 회사의 finetuning 예정 데이터셋과 대조해보기

jsonl 데이터를 판다스로 불러오기

In [ ]:
filename = "lamini_docs.jsonl"
instruction_dataset_df = pd.read_json(filename, lines=True)
instruction_dataset_df

### 데이터를 구성하는 다양한 방식들

가장 간단한 방식 중 하나는 질문과 답변을 이어붙이는 것입니다

In [ ]:
examples = instruction_dataset_df.to_dict()
text = examples["question"][0] + examples["answer"][0]
text

In [ ]:
if "question" in examples and "answer" in examples:
  text = examples["question"][0] + examples["answer"][0]
elif "instruction" in examples and "response" in examples:
  text = examples["instruction"][0] + examples["response"][0]
elif "input" in examples and "output" in examples:
  text = examples["input"][0] + examples["output"][0]
else:
  text = examples["text"][0]

Q & A 탬플릿을 생성합니다

In [ ]:
prompt_template_qa = """### Question:
{question}

### Answer:
{answer}"""

In [ ]:
question = examples["question"][0]
answer = examples["answer"][0]

text_with_prompt_template = prompt_template_qa.format(question=question, answer=answer)
text_with_prompt_template

In [ ]:
prompt_template_q = """### Question:
{question}

### Answer:"""

In [ ]:
num_examples = len(examples["question"])
finetuning_dataset_text_only = []
finetuning_dataset_question_answer = []
for i in range(num_examples):
  question = examples["question"][i]
  answer = examples["answer"][i]

  text_with_prompt_template_qa = prompt_template_qa.format(question=question, answer=answer)
  finetuning_dataset_text_only.append({"text": text_with_prompt_template_qa})

  text_with_prompt_template_q = prompt_template_q.format(question=question)
  finetuning_dataset_question_answer.append({"question": text_with_prompt_template_q, "answer": answer})

pprint를 활용하면 딕셔너리 데이터를 편하게 확인할 수 있습니다

In [ ]:
pprint(finetuning_dataset_text_only[0])

In [ ]:
pprint(finetuning_dataset_question_answer[0])

### 데이터를 저장하는 일반적인 방식

In [ ]:
with jsonlines.open(f'lamini_docs_processed.jsonl', 'w') as writer:
    writer.write_all(finetuning_dataset_question_answer)

In [ ]:
finetuning_dataset_name = "lamini/lamini_docs"
finetuning_dataset = load_dataset(finetuning_dataset_name)
print(finetuning_dataset)